<a href="https://colab.research.google.com/github/Dawudis/Political-Web-Scraping-Project/blob/main/Scraping%2BTokenization%2BSentimentAnalysis%2BNER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install/Import Dependencies**

In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers requests beautifulsoup4 pandas numpy

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [49]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
!pip install newspaper3k

import newspaper
from newspaper import Article

In [ ]:
!pip install nltk

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [ ]:
!pip install -U spacy
!python -m spacy download en

import spacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [53]:
import numpy as np
import pandas as pd

# **Scrape Article**

In [54]:
url = "https://apnews.com/article/biden-congress-government-overhaul-public-works-9438a78ba7c7865d75428e746d1de04d"

In [64]:
article = Article(url)
article.download()
article.parse()
print(article.text)

President Joe Biden walks over to speak with members of the press before boarding Marine One on the South Lawn of the White House, Saturday, Oct. 2, 2021, in Washington. Biden is spending the weekend at his home in Delaware. (AP Photo/Patrick Semansky)

President Joe Biden walks over to speak with members of the press before boarding Marine One on the South Lawn of the White House, Saturday, Oct. 2, 2021, in Washington. Biden is spending the weekend at his home in Delaware. (AP Photo/Patrick Semansky)

WASHINGTON (AP) — President Joe Biden on Saturday acknowledged frustrations as Democrats strain to rescue a scaled-back version of his $3.5 trillion government-overhaul plan and salvage a related public works bill after frantic negotiations failed to produce a deal.

“Everybody’s frustrated, it’s part of being in government, being frustrated,” Biden told reporters before leaving the White House for a weekend stay at his home in Wilmington, Delaware. He pledged to ”work like hell” to get 

# **Tokenize Article into Sentences**

In [56]:
sentences = sent_tokenize(article.text)

# **Configure DataFrame and Apply Sentiment Analysis + NER**

In [57]:
df = pd.DataFrame(np.array(sentences), columns=['article sentences'])

In [58]:
def sentiment_score(sentences):
    tokens = tokenizer.encode(sentences, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [59]:
df['sentiment'] = df['article sentences'].apply(lambda x: sentiment_score(x[:512]))

In [61]:
df['named entities'] = df['article sentences'].apply(lambda x: list(nlp(x).ents))

In [63]:
df

,article sentences,sentiment,named entities
0,President Joe Biden walks over to speak with m...,1,"[(Joe, Biden), (the, South, Lawn), (the, White..."
1,Biden is spending the weekend at his home in D...,4,"[(Biden), (the, weekend), (Delaware)]"
2,(AP Photo/Patrick Semansky)\n\nPresident Joe B...,3,"[(AP, Photo, /, Patrick, Semansky), (Joe, Bide..."
3,Biden is spending the weekend at his home in D...,4,"[(Biden), (the, weekend), (Delaware)]"
4,(AP Photo/Patrick Semansky)\n\nWASHINGTON (AP)...,1,"[(AP, Photo, /, Patrick, Semansky), (WASHINGTO..."
5,"“Everybody’s frustrated, it’s part of being in...",1,"[(Biden), (the, White, House), (a, weekend), (..."
6,He pledged to ”work like hell” to get the two ...,1,[(two)]
7,The president had gone to Capitol Hill on Frid...,2,"[(Capitol, Hill), (Friday), (House), (Democrats)]"
8,"According to lawmakers in the room, he discuss...",1,"[($, 1.9, trillion)]"
9,ADVERTISEMENT\n\nThe White House and its allie...,4,"[(The, White, House), (Congress)]"
